<a href="https://colab.research.google.com/github/agrezende/dask-tutorial/blob/develop/00_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup inicial para utilização do Google Colab

Para a utilização dos notebooks no Google Colab, algumas configurações são necessárias.

Reproduzir os comandos desse notebook em todos aqueles que for necessário alguma preparação.

Para utilização em ambiente Jupyter Lab, outros ajustes são necessários.

In [ ]:
# Devido a varios erros encontrados, eh necessario realizar o upgrade de versao do pacotes:
# dask, distributed
# distributed==1.25.3
# cloudpickle (sera atualizado como dependencia)
# Em 16/03/2021:
# < cloudpickle==1.3.0
# > cloudpickle==1.6.0
# < dask==2.12.0
# > dask==2021.3.0
# < distributed==1.25.3
# > distributed==2021.3.0

!pip freeze > antes.txt
!pip install dask distributed --upgrade
!pip freeze > depois.txt
!echo Alteracoes:
# !diff antes.txt depois.txt && echo a || echo b
!diff --side-by-side --suppress-common-lines antes.txt depois.txt && echo Nao houve alteracoes
!rm antes.txt depois.txt

Requirement already up-to-date: dask in /usr/local/lib/python3.7/dist-packages (2021.3.0)
Requirement already up-to-date: distributed in /usr/local/lib/python3.7/dist-packages (2021.3.0)
Alteracoes:
Nao houve alteracoes


In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

# client = Client(processes=False, threads_per_worker=4, n_workers=1, memory_limit='2GB', scheduler_port=8780)

client

/usr/local/lib/python3.7/dist-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34639 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:41825 Dashboard: http://127.0.0.1:34639/status,Cluster Workers: 4 Cores: 4 Memory: 13.65 GB


In [ ]:
!git clone --single-branch --branch develop https://github.com/agrezende/dask-tutorial.git

Cloning into 'dask-tutorial'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 2673 (delta 130), reused 162 (delta 106), pack-reused 2452
Receiving objects: 100% (2673/2673), 84.09 MiB | 24.24 MiB/s, done.
Resolving deltas: 100% (1720/1720), done.


In [ ]:
import os
!pwd
os.chdir('/content/dask-tutorial')
!pwd

/content
/content/dask-tutorial


In [ ]:
%run prep.py -d flights

- Downloading NYC Flights dataset... done
- Extracting flight data... done
- Creating json data... done
** Created flights dataset! in 5.04s**


In [ ]:
from glob import glob
# filenames = sorted(glob(os.path.join('data', 'nycflights', '*.csv')))

# Para funcionar corretamente com a leitura atradada, eh necessario o path completo:
filenames = sorted(glob(os.path.join('/content/dask-tutorial/data', 'nycflights', '*.csv')))